In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV  # Para Brier Score
from sklearn.metrics import roc_auc_score, brier_score_loss, classification_report
import joblib

# --- 1. Definición de Features y Target ---
# Como discutimos, excluimos las "fugas de datos" (SIT_FIN, PROM_GRAL, etc.)
TARGET = "Desercion"
FEATURES = [
    'COD_ENSE',
    'NOM_RBD',
    'COD_JOR',
    'COD_GRADO',
    'NOM_COM_RBD',
    'NOM_COM_ALU',
    'NOM_DEPROV_RBD',
    'NOM_REG_RBD_A',
    'COD_DEPE'
]



In [ ]:
df_2018 = pd.read_csv("/content/dataset_balanceado_2018.csv", delimiter=';')
df_2019 = pd.read_csv("/content/dataset_balanceado_2019.csv", delimiter=';')

In [ ]:
df_2018[FEATURES].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67464 entries, 0 to 67463
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   COD_ENSE        67463 non-null  object
 1   NOM_RBD         67464 non-null  object
 2   COD_JOR         67463 non-null  object
 3   COD_GRADO       67463 non-null  object
 4   NOM_COM_RBD     67464 non-null  object
 5   NOM_COM_ALU     67463 non-null  object
 6   NOM_DEPROV_RBD  67464 non-null  object
 7   NOM_REG_RBD_A   67464 non-null  object
 8   COD_DEPE        67464 non-null  object
dtypes: object(9)
memory usage: 4.6+ MB


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
# joblib ya está importado en tu primera celda

# --- (Aquí cargas tus df_2018 y df_2019) ---

# 1. Define tu lista de features de TEXTO (object)
# (Esta es la lista que sacaste del .info() en el paso anterior)
categorical_features = [
    'COD_ENSE',
    'NOM_RBD',
    'COD_JOR',
    'COD_GRADO',
    'NOM_COM_RBD',
    'NOM_COM_ALU',
    'NOM_DEPROV_RBD',
    'NOM_REG_RBD_A',
    'COD_DEPE'
]


# (Asegúrate de que tus 'COD_' SÍ sean numéricos. Si 'COD_ENSE'
#  te salió como 'object', agrégalo a la lista de arriba)


# 2. Crear el Codificador (Encoder)
# handle_unknown='use_encoded_value' y unknown_value=-1
# son claves para que no falle con los datos de 2019.
encoder = OrdinalEncoder(
    handle_unknown='use_encoded_value',
    unknown_value=-1  # Marcará categorías nuevas (de 2019) como -1
)


# 3. Aprender (FIT) las categorías SÓLO con datos de 2018
print("Ajustando el codificador con datos de 2018...")
# (Asegúrate de limpiar nulos ANTES, si no lo has hecho)
# df_2018[categorical_features] = df_2018[categorical_features].fillna('MISSING')
# df_2019[categorical_features] = df_2019[categorical_features].fillna('MISSING')

encoder.fit(df_2018[categorical_features])

# --- ¡¡AQUÍ!! ESTE ES EL CÓDIGO QUE DEBES AÑADIR ---
print("Guardando el encoder en 'encoder.pkl'...")
joblib.dump(encoder, "encoder.pkl")
print("¡Encoder guardado!")
# ----------------------------------------------------


# 4. Transformar AMBOS dataframes (2018 y 2019)
print("Transformando 2018 y 2019 a números...")
df_2018[categorical_features] = encoder.transform(df_2018[categorical_features])
df_2019[categorical_features] = encoder.transform(df_2019[categorical_features])


# --- ¡LISTO! ---
print("\n¡Transformación completada!")
print("Tus columnas de texto ahora son numéricas.")

# Ahora, cuando definas tu X_train, ya será numérico:
# X_train = df_2018[FEATURES]
# y_train = df_2018[TARGET]
# ... (etc)

Ajustando el codificador con datos de 2018...
Guardando el encoder en 'encoder.pkl'...
¡Encoder guardado!
Transformando 2018 y 2019 a números...

¡Transformación completada!
Tus columnas de texto ahora son numéricas.


In [ ]:
df_2018[FEATURES].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172656 entries, 0 to 172655
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   COD_ENSE        172656 non-null  float64
 1   NOM_RBD         172656 non-null  float64
 2   COD_JOR         172656 non-null  float64
 3   COD_GRADO       172656 non-null  float64
 4   NOM_COM_RBD     172656 non-null  float64
 5   NOM_COM_ALU     172656 non-null  float64
 6   NOM_DEPROV_RBD  172656 non-null  float64
 7   NOM_REG_RBD_A   172656 non-null  float64
 8   COD_DEPE        172656 non-null  float64
dtypes: float64(9)
memory usage: 11.9 MB


In [ ]:
df_2019[FEATURES].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150366 entries, 0 to 150365
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   COD_ENSE        150366 non-null  float64
 1   NOM_RBD         150366 non-null  float64
 2   COD_JOR         150366 non-null  float64
 3   COD_GRADO       150366 non-null  float64
 4   NOM_COM_RBD     150366 non-null  float64
 5   NOM_COM_ALU     150366 non-null  float64
 6   NOM_DEPROV_RBD  150366 non-null  float64
 7   NOM_REG_RBD_A   150366 non-null  float64
 8   COD_DEPE        150366 non-null  float64
dtypes: float64(9)
memory usage: 10.3 MB


In [ ]:
df_2018

,AGNO,NOM_RBD,NOM_REG_RBD_A,NOM_COM_RBD,NOM_DEPROV_RBD,COD_DEPE,COD_DEPE2,RURAL_RBD,ESTADO_ESTAB,COD_ENSE,COD_GRADO,COD_JOR,MRUN,GEN_ALU,FEC_NAC_ALU,NOM_COM_ALU,SIT_FIN,SIT_FIN_R,Desercion
0,2018,COLEGIO LIDIA GONZALEZ BARRIGA,ARAUC,COLLIPULLI,MALLECO,Particular Subvencionado,Particular Subvencionado,Urbano,Funcionando,Enseñanza Básica,4º Básico,Mañana y tarde,3852981,Femenino,200811,TALTAL,Retirado,Trasladado,0
1,2018,LICEO COLEGIO DEPORTIVO,TPCA,IQUIQUE,IQUIQUE,Corporación Municipal,Municipal,Urbano,Funcionando,Enseñanza Básica,3º Básico,Mañana,25710531,Masculino,200812,IQUIQUE,Retirado,Trasladado,0
2,2018,LICEO MUNICIPAL DE BATUCO,RM,LAMPA,SANTIAGO NORTE,Corporación Municipal,Municipal,Urbano,Funcionando,Media Humanístico-Científica (jóvenes),3º Medio,Mañana y tarde,17212012,Femenino,200106,LAMPA,Retirado,Retirado,1
3,2018,ESCUELA BÁSICA LIBERTAD,VALPO,SAN ESTEBAN,LOS ANDES,Municipal DAEM,Municipal,Urbano,Funcionando,Enseñanza Básica,5º Básico,Mañana y tarde,4990910,Femenino,200712,SAN ESTEBAN,Promovido,Promovido,0
4,2018,COLEGIO AMERICA,ARAUC,VILCÚN,CAUTÍN NORTE,Municipal DAEM,Municipal,Urbano,Funcionando,Media Humanístico-Científica (jóvenes),2º Medio,Mañana y tarde,13370164,Femenino,200305,VILCÚN,Promovido,Promovido,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172651,2018,ESCUELA BASICA SANTA ROSA DEL HUERTO,VALPO,CALERA,QUILLOTA,Municipal DAEM,Municipal,Urbano,Funcionando,Enseñanza Básica,7º Básico,Mañana y tarde,10908531,Masculino,200602,CALERA,Promovido,Promovido,0
172652,2018,COLEGIO ACONCAGUA,VALPO,QUILPUÉ,VALPARAÍSO,Particular Subvencionado,Particular Subvencionado,Urbano,Funcionando,Media Humanístico-Científica (jóvenes),2º Medio,Mañana y tarde,5321760,Masculino,200305,VILLA ALEMANA,Promovido,Promovido,0
172653,2018,LICEO POLIVALENTE PEDRO DEL RÍO ZAÑARTU,BBIO,HUALPÉN,CONCEPCIÓN,Municipal DAEM,Municipal,Urbano,Funcionando,Media TP Técnica (jóvenes),3º Medio,Mañana,15017744,Femenino,200002,HUALPÉN,Promovido,Promovido,0
172654,2018,COLEGIO SAN FRANCISCO DE ASIS,LAGOS,CASTRO,CHILOÉ,Particular Subvencionado,Particular Subvencionado,Urbano,Funcionando,Media Humanístico-Científica (jóvenes),2º Medio,Mañana y tarde,16129607,Femenino,200204,CHILE CHICO,Promovido,Promovido,0


In [ ]:
df_2019

,AGNO,NOM_RBD,NOM_REG_RBD_A,NOM_COM_RBD,NOM_DEPROV_RBD,COD_DEPE,COD_DEPE2,RURAL_RBD,ESTADO_ESTAB,COD_ENSE,COD_GRADO,COD_JOR,MRUN,GEN_ALU,FEC_NAC_ALU,NOM_COM_ALU,SIT_FIN,SIT_FIN_R,Desercion
0,2019,INSTITUTO SAN VICENTE DE TAGUA TAGUA,LGBO,SAN VICENTE,CACHAPOAL,Corporación Municipal,Municipal,Urbano,Funcionando,Media Humanístico-Científica (jóvenes),1º Medio,Mañana y tarde,23358786,Masculino,200308,SAN VICENTE,Retirado,Retirado,1
1,2019,ESCUELA AGRICOLA DE LA PATAGONIA,AYSEN,COYHAIQUE,COYHAIQUE,Corporación Adm. Delegada,Corporación Adm. Delegada,Urbano,Funcionando,Media TP Industrial (jóvenes),2º Medio,Tarde,21638410,Masculino,200104,CISNES,Retirado,Retirado,1
2,2019,COLEGIO POLITEC.NTRA.SRA.DE LA PRESENTAC,RM,MELIPILLA,TALAGANTE,Particular Subvencionado,Particular Subvencionado,Urbano,Funcionando,Media TP Industrial (jóvenes),2º Medio,Mañana y tarde,25840676,Masculino,200307,MELIPILLA,Retirado,Retirado,1
3,2019,COLEGIO MAYOR DE PEÑALOLEN,RM,PEÑALOLÉN,SANTIAGO ORIENTE,Particular Pagado,Particular Pagado,Urbano,Funcionando,Media Humanístico-Científica (jóvenes),1º Medio,Mañana y tarde,14488017,Masculino,200407,MACUL,Promovido,Promovido,0
4,2019,ESCUELA N°1124 ALIANZA AMERICANA,RM,QUINTA NORMAL,SANTIAGO PONIENTE,Particular Subvencionado,Particular Subvencionado,Urbano,Funcionando,Enseñanza Básica,1º Básico,Tarde,2630939,Masculino,201209,QUINTA NORMAL,Promovido,Promovido,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150361,2019,LICEO LOS ÁNGELES A-59,BBIO,LOS ÁNGELES,BIOBÍO,Municipal DAEM,Municipal,Urbano,Funcionando,Media Humanístico-Científica (jóvenes),3º Medio,Mañana y tarde,18643406,Femenino,200209,LOS ÁNGELES,Promovido,Promovido,0
150362,2019,COLEGIO SAN JUAN DIEGO,RM,RECOLETA,SANTIAGO NORTE,Particular Subvencionado,Particular Subvencionado,Urbano,Funcionando,Enseñanza Básica,5º Básico,Mañana,5416264,Femenino,200810,RECOLETA,Promovido,Promovido,0
150363,2019,COLEGIO CASTELGANDOLFO,RM,PADRE HURTADO,TALAGANTE,Particular Pagado,Particular Pagado,Urbano,Funcionando,Media Humanístico-Científica (jóvenes),3º Medio,Mañana y tarde,20289035,Femenino,200305,MAIPÚ,Promovido,Promovido,0
150364,2019,COLEGIO HISPANO AMERICANO,VALPO,VIÑA DEL MAR,VALPARAÍSO,Particular Subvencionado,Particular Subvencionado,Urbano,Funcionando,Media TP Comercial (jóvenes),4º Medio,Mañana y tarde,5115881,Masculino,200206,VIÑA DEL MAR,Promovido,Promovido,0


In [ ]:
# --- 3. Validación Temporal Obligatoria ---
X_train = df_2018[FEATURES]
y_train = df_2018[TARGET]

X_test = df_2019[FEATURES]
y_test = df_2019[TARGET]

print(f"Entrenando con {len(X_train)} registros (2018)")
print(f"Probando con {len(X_test)} registros (2019)")

Entrenando con 172656 registros (2018)
Probando con 150366 registros (2019)


In [ ]:
# --- 4. Entrenamiento del Modelo ---
# (Usamos RandomForest como base, como querías)
print("Entrenando RandomForest...")
modelo_rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42, # ¡Seed obligatorio por la rúbrica!
    n_jobs=-1
)
modelo_rf.fit(X_train, y_train)

Entrenando RandomForest...


RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [ ]:
### 5. Entrenamiento del Modelo (¡AHORA CALIBRADO!)
# ----------------------------------------------------

# (Tus X_train, y_train ya están definidos desde el Paso 4)

# 1. Define tu modelo base (PERO NO LO ENTRENES AÚN)
modelo_rf_base = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

# 2. Define el Calibrador
#    cv=3: Usa validación cruzada (sobre X_train) para generar
#          predicciones "limpias" y entrenar el calibrador.
#    method='isotonic': Es el método más potente.
print("Entrenando y Calibrando el modelo (esto puede tardar)...")

calibrated_model = CalibratedClassifierCV(
    modelo_rf_base,
    method='isotonic',
    cv=3  # 3 folds es más rápido que 5
)

# 3. Entrena el modelo calibrado
calibrated_model.fit(X_train, y_train)

Calibrando modelo...


/usr/local/lib/python3.12/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


CalibratedClassifierCV(cv='prefit',
                       estimator=RandomForestClassifier(n_estimators=200,
                                                        n_jobs=-1,
                                                        random_state=42),
                       method='isotonic')

In [ ]:
### 6. Evaluación (Métricas de la Rúbrica)
# ----------------------------------------------------

print("Evaluando en datos de 2019...")

# ¡IMPORTANTE! Usa el 'calibrated_model' para predecir,
# NO el 'modelo_rf_base'.
y_pred_proba = calibrated_model.predict_proba(X_test)[:, 1]
y_pred_class = calibrated_model.predict(X_test)

# Métricas obligatorias
auc = roc_auc_score(y_test, y_pred_proba)
brier = brier_score_loss(y_test, y_pred_proba) # <-- ESTE DEBERÍA BAJAR

print("\n--- MÉTRICAS DE EVALUACIÓN (TEST 2019) ---")
print(f"🎯 AUROC (Métrica Principal): {auc:.4f}")
print(f"🔥 Brier Score (Calibración): {brier:.4f}") # Objetivo: < 0.12

print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_class))

### 7. Guardar el Modelo (para la API)
# ----------------------------------------------------
print("Guardando modelo CALIBRADO...")
joblib.dump(calibrated_model, "modelo_desercion.pkl")

Evaluando en datos de 2019...

--- MÉTRICAS DE EVALUACIÓN (TEST 2019) ---
🎯 AUROC (Métrica Principal): 0.8133
🔥 Brier Score (Calibración): 0.1958

Reporte de Clasificación (para F1/Recall):
              precision    recall  f1-score   support

           0       0.73      0.80      0.76     75183
           1       0.77      0.70      0.74     75183

    accuracy                           0.75    150366
   macro avg       0.75      0.75      0.75    150366
weighted avg       0.75      0.75      0.75    150366



In [ ]:
# Celda para instalar SHAP (necesario en Colab)
!pip install shap

In [ ]:
# --- 7. Guardar el Modelo (para la API) ---
print("Guardando modelo...")
# Guardamos el modelo YA CALIBRADO
joblib.dump(calibrated_model, "modelo_desercion.pkl")

# NO necesitas un "vectorizer.pkl" porque no estás usando NLP.
# Tu API simplemente recibirá un JSON con los valores de las FEATURES.

print("¡Proceso de ML completado!")

Guardando modelo...
¡Proceso de ML completado!


In [ ]:
import os

file_path = "modelo_desercion.pkl"
file_size = os.path.getsize(file_path)
print(f"El archivo '{file_path}' pesa {file_size} bytes.")

El archivo 'modelo_desercion.pkl' pesa 992343578 bytes.
